### ANALIZANDO COVID DATASET

In [1]:
import pandas as pd

In [2]:
mi_data_covid = pd.read_csv("data/fallecidos_covid.csv", delimiter=";", encoding="utf-8",  parse_dates=["FECHA_CORTE", "FECHA_FALLECIMIENTO"])
mi_data_covid.head()

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID
0,2024-03-17,2022-02-19,63,MASCULINO,Criterio virológico,TUMBES,TUMBES,TUMBES,240101.0,203506.0
1,2024-03-17,2021-05-29,74,MASCULINO,Criterio virológico,TUMBES,TUMBES,TUMBES,240101.0,203532.0
2,2024-03-17,2021-06-23,72,FEMENINO,Criterio SINADEF,TACNA,TACNA,ALTO DE LA ALIANZA,230102.0,203584.0
3,2024-03-17,2021-08-24,85,MASCULINO,Criterio investigación Epidemiológica,TUMBES,TUMBES,TUMBES,240101.0,212541.0
4,2024-03-17,2021-06-27,46,MASCULINO,Criterio virológico,TACNA,TACNA,ALTO DE LA ALIANZA,230102.0,203665.0


### Verificar Valores únicos
Para identificar posibles errores o valores inusuales en los nombres de los departamentos, se listaron todos los valores únicos presentes en la columna "DEPARTAMENTO":  
Para ello usaremos la funcion unique().

In [3]:
mi_data_covid["DEPARTAMENTO"].unique()

array(['TUMBES', 'TACNA', 'PIURA', 'PUNO', 'UCAYALI', 'SAN MARTIN',
       'LIMA', 'CUSCO', 'CAJAMARCA', 'AREQUIPA', 'LA LIBERTAD',
       'AMAZONAS', 'LORETO', 'MOQUEGUA', 'MADRE DE DIOS', 'LAMBAYEQUE',
       'APURIMAC', 'PASCO', 'ICA', 'HUANUCO', 'ANCASH', 'JUNIN', 'CALLAO',
       'AYACUCHO', 'HUANCAVELICA', 'LIMA METROPOLITANA', 'MASCULINO'],
      dtype=object)

### Identificación de Inconsistencias 

Hemos encontraro dos casos problemáticos en los nombres de departamentos:


1. LIMA METROPOLITANA:  Este valor debemos reemplazardo por "LIMA" para mantener la consistencia en los datos.  
2. MASCULINO: Este valor es incorrecto, ya que no representa un departamento. Se debía identificar a qué departamento pertenece este registro y corregirlo.  


Para ello lo que haremos es verificar cuantos registros encontramos con el nombre LIMA METROPOLITANA.

In [4]:
mi_data_covid[mi_data_covid["DEPARTAMENTO"] == "LIMA METROPOLITANA"]

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID
6792,2024-03-17,2024-01-19,91,FEMENINO,NotiCovid,LIMA METROPOLITANA,LIMA,SAN LUIS,150134.0,998659.0
12138,2024-03-17,2024-01-06,92,FEMENINO,NotiCovid,LIMA METROPOLITANA,LIMA,COMAS,150110.0,1495320.0
39209,2024-03-17,2024-01-14,83,MASCULINO,NotiCovid,LIMA METROPOLITANA,CAÑETE,SAN LUIS,150514.0,5167332.0
40209,2024-03-17,2024-01-08,82,FEMENINO,NotiCovid,LIMA METROPOLITANA,LIMA,SAN JUAN DE MIRAFLORES,150133.0,5401073.0
47651,2024-03-17,2024-01-06,90,FEMENINO,NotiCovid,LIMA METROPOLITANA,LIMA,PACHACAMAC,150123.0,7357257.0
69365,2024-03-17,2024-01-12,76,MASCULINO,NotiCovid,LIMA METROPOLITANA,CAÑETE,IMPERIAL,150507.0,13614490.0
84535,2024-03-17,2024-01-07,68,MASCULINO,NotiCovid,LIMA METROPOLITANA,LIMA,SAN JUAN DE MIRAFLORES,150133.0,19253071.0
85686,2024-03-17,2024-01-03,65,MASCULINO,NotiCovid,LIMA METROPOLITANA,LIMA,LOS OLIVOS,150117.0,20100022.0


### Caso 1: Reemplazo de "LIMA METROPOLITANA" por "LIMA"
Utilizaremos la función replace() de pandas para realizar el reemplazo directo en la columna  "DEPARTAMENTO":

In [5]:
mi_data_covid["DEPARTAMENTO"] = mi_data_covid["DEPARTAMENTO"].replace("LIMA METROPOLITANA", "LIMA")
#Verificar si exista LIMA METROPOLITANA
#Si el reemplazo se hizo correctamente, no deberia mostrar ningun resultado la siguiente comparación.
mi_data_covid[mi_data_covid["DEPARTAMENTO"] == "LIMA METROPOLITANA"]

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID


#### Caso 2: Corrección de "MASCULINO" en la Columna "DEPARTAMENTO"

Primero, identificaremos los registros específicos donde "DEPARTAMENTO" es  igual a "MASCULINO":

In [6]:
mi_data_covid[mi_data_covid["DEPARTAMENTO"]=="MASCULINO"]

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID
7946,2024-03-17,2024-01-20,86,MASCULINO,NotiCovid,MASCULINO,TACNA,TACNA,230101.0,117420.0
73412,2024-03-17,2024-01-21,81,MASCULINO,NotiCovid,MASCULINO,HUANCAVELICA,PALCA,90114.0,15162227.0


Detectamos que son 2 registros y podemos hacer las modificaciones en cada caso: 

1. Hacemos el filtro con el campo UUID usando su codigo respectivo.
2. Seguido llamamos a la columna DEPARTAMENTO, para poder hacer el replace de MASCULINO.
3. En cada caso lo reemplazamos por el DEPARTAMENTO QUE LE CORRESPONDE.
4. Como sabemos que departamento es el que le corresponde:  
    4.1. Guiandonos por la columna PROVINCIA, en este caso la provincia de cada caso tiene el mismo nombre de su DEPARTAMENTO.

Para localizar los registros usaremos .loc, ya que nos permite seleccionar las filas en base a las condiciones que usamos.

In [7]:
mi_data_covid.loc[mi_data_covid["UUID"] == 117420, "DEPARTAMENTO"]  = mi_data_covid.loc[mi_data_covid["UUID"] == 117420]["DEPARTAMENTO"].replace("MASCULINO", "TACNA")
mi_data_covid.loc[mi_data_covid["UUID"] == 15162227, "DEPARTAMENTO"]  = mi_data_covid.loc[mi_data_covid["UUID"] ==15162227]["DEPARTAMENTO"].replace("MASCULINO", "HUANCAVELICA")

mi_data_covid[mi_data_covid["UUID"].isin([117420, 15162227]) ]

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID
7946,2024-03-17,2024-01-20,86,MASCULINO,NotiCovid,TACNA,TACNA,TACNA,230101.0,117420.0
73412,2024-03-17,2024-01-21,81,MASCULINO,NotiCovid,HUANCAVELICA,HUANCAVELICA,PALCA,90114.0,15162227.0


### CORRIGIENDO FECHAS

Si bien pudieron visualizar al inicio del proyecto al momento de leer el archivo csv, se incluyó un parametro llamado parse_dates al cual se le pasaron las columnas que contienen fechas.  
Bueno esto se hace con el fin de poder asignarle el formato fecha a esos campos, ya que si los traemos directamente sin especificarles ese parse_dates, los datos seran tomados como string.  
En este caso en particular si revisamos el archivo ambas columnas vienen de la siguiente forma:  

 
|  |Fecha_Corte |Fecha_Fallecimiento |
|--|------------|--------------------|
|0 | 20240317   |      20200307      |
|1 | 20240317   |      20220317      |
|2 | 20240317   |      20210611      |
|3 | 20240317   |      20230512      |
|4 | 20240317   |      20210122      |


Pero al especificar el parse_dates y poner las columnas que corresponde a las fechas ya se corrige

### COLUMNA SEXO:

Revisemos si existen los 2 tipos de sexos, si hay algun campo vacio, o mal escrito, podremos corregirlo.  
Para ello usamos la funcion `unique()`

In [8]:
mi_data_covid["SEXO"].unique()

array(['MASCULINO', 'FEMENINO'], dtype=object)

### COLUMNA CLASIFICACIÓN:

Podemos usar esta columna para saber los tipos de clasificación que cuenta esta base y poder saber el total de cada uno de ellos.

In [9]:
mi_data_covid["CLASIFICACION_DEF"].value_counts()

CLASIFICACION_DEF
Criterio virológico                      84557
Criterio SINADEF                         66198
Criterio serológico                      42910
Criterio investigación Epidemiológica    10783
Criterio clínico                          8994
Criterio radiológico                      5113
Criterio nexo epidemiológico              2215
NotiCovid                                  132
sinadef                                     16
Name: count, dtype: int64

### COLUMNA EDAD:

Aquí veremos si existen edades fuera del rango general.  

1. Primero validemos si existen edad menor o igual a 0

In [10]:
mi_data_covid[mi_data_covid["EDAD_DECLARADA"]<= 0]

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID
6093,2024-03-17,2020-07-25,0,FEMENINO,Criterio nexo epidemiológico,LA LIBERTAD,ASCOPE,PAIJAN,130205.0,13412.0
7226,2024-03-17,2020-07-09,0,MASCULINO,Criterio nexo epidemiológico,HUANUCO,HUANUCO,SANTA MARIA DEL VALLE,100109.0,16495.0
12075,2024-03-17,2022-02-10,0,MASCULINO,Criterio virológico,AYACUCHO,HUAMANGA,AYACUCHO,50101.0,29620128.0
26223,2024-03-17,2021-06-03,0,FEMENINO,Criterio SINADEF,LIMA,LIMA,EL AGUSTINO,150111.0,3156285.0
48715,2024-03-17,2020-05-15,0,MASCULINO,Criterio SINADEF,LAMBAYEQUE,LAMBAYEQUE,MORROPE,140306.0,23999393.0
...,...,...,...,...,...,...,...,...,...,...
220531,2024-03-17,2022-02-12,0,MASCULINO,Criterio SINADEF,MADRE DE DIOS,TAMBOPATA,TAMBOPATA,170101.0,NaN
220551,2024-03-17,2021-07-04,0,FEMENINO,Criterio SINADEF,LIMA,LIMA,SAN MARTIN DE PORRES,150135.0,NaN
220810,2024-03-17,2020-04-26,0,FEMENINO,Criterio SINADEF,CALLAO,CALLAO,BELLAVISTA,70102.0,NaN
220817,2024-03-17,2021-04-11,0,MASCULINO,Criterio nexo epidemiológico,LA LIBERTAD,TRUJILLO,TRUJILLO,130101.0,NaN


2. Veamos el total que tienen por cada uno en este caso cuantas personas tienen asignado la Edad 0 

In [11]:

print(mi_data_covid.loc[mi_data_covid["EDAD_DECLARADA"] <= 0]["EDAD_DECLARADA"].value_counts())


EDAD_DECLARADA
0    257
Name: count, dtype: int64


3. Y veamos el total de personas tienen asignado la Edad > 100 

In [12]:

mi_data_covid.loc[mi_data_covid["EDAD_DECLARADA"] > 100]["EDAD_DECLARADA"].value_counts()

EDAD_DECLARADA
101    101
102     66
103     33
104     26
106     14
105      9
108      7
107      5
110      3
113      2
109      2
114      1
Name: count, dtype: int64

### Total de Fallecidos por Departamento

Para calcular el total de personas fallecidas por departamento, podemos utilizar la función `groupby()` de Pandas. 

1. **Agrupación por Departamento**:  
   Usamos el campo correspondiente al departamento como clave de agrupación.

2. **Conteo de Fallecidos**:  
   Posteriormente, aplicamos la función de conteo sobre el campo único (por ejemplo, `UUID`) para obtener el número total de registros por cada grupo.

El código tendría una estructura similar a esta:

```python
df.groupby("Departamento")["UUID"].count()


In [13]:
mi_data_covid.groupby("DEPARTAMENTO")["UUID"].count()

DEPARTAMENTO
AMAZONAS          1391
ANCASH            7363
APURIMAC          1696
AREQUIPA         10636
AYACUCHO          2423
CAJAMARCA         4615
CALLAO           10853
CUSCO             5291
HUANCAVELICA      1297
HUANUCO           2951
ICA               9084
JUNIN             7602
LA LIBERTAD      11005
LAMBAYEQUE        9294
LIMA             96469
LORETO            4352
MADRE DE DIOS      884
MOQUEGUA          1709
PASCO             1131
PIURA            13101
PUNO              4836
SAN MARTIN        3228
TACNA             2265
TUMBES            1756
UCAYALI           3217
Name: UUID, dtype: int64

### VALIDAR COLUMNAS CON DATOS NULOS

Con la siguiente funcion podemos saber de manera general de todas las columnas cuales son las que tendran valores nulos.
usando la funcion `isnull()` y luego aplicando la funcion `sum()` sabremos de manera general que columnas tendran valores nulos y esto nos lo mostrara en la columna total
los que tengan totales mayores a 0 son las columnas con valores nulos.


In [14]:
mi_data_covid.isnull().sum()

FECHA_CORTE               0
FECHA_FALLECIMIENTO       0
EDAD_DECLARADA            0
SEXO                      0
CLASIFICACION_DEF         0
DEPARTAMENTO              0
PROVINCIA                 5
DISTRITO                  5
UBIGEO                    5
UUID                   2469
dtype: int64

### Verificación de Valores Nulos por Departamento

Con los datos detectados, podemos realizar algunas verificaciones clave para asegurar la integridad de los cálculos.

#### Paso 1: Contar los Valores por Departamento
Podemos comprobar cuántos valores son nulos y cuántos no en cada departamento.  
Los valores que **no son nulos** deberían coincidir con el total calculado usando la siguiente función:

```python
mi_data_covid.groupby("DEPARTAMENTO")["UUID"].count()
```

Si los resultados coinciden, confirmamos que la función está filtrando correctamente y no está contando los campos nulos.

#### Paso 2: Identificar los Campos Nulos
Para realizar una revisión más detallada:

Buscamos los registros nulos en la columna UUID.
Elegimos un departamento específico para verificar manualmente si los resultados coinciden con el cálculo anterior.

In [15]:
mi_data_covid[mi_data_covid["UUID"].isnull()]

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID
218449,2024-03-17,2021-06-06,58,MASCULINO,Criterio SINADEF,AREQUIPA,AREQUIPA,PAUCARPATA,40112.0,NaN
218450,2024-03-17,2021-03-13,28,MASCULINO,Criterio serológico,PUNO,CHUCUITO,DESAGUADERO,210402.0,NaN
218451,2024-03-17,2020-10-14,68,MASCULINO,Criterio investigación Epidemiológica,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101.0,NaN
218452,2024-03-17,2020-06-25,0,MASCULINO,Criterio SINADEF,LIMA,CAÑETE,SAN VICENTE DE CAÑETE,150501.0,NaN
218453,2024-03-17,2021-04-04,53,FEMENINO,Criterio virológico,LIMA,LIMA,CARABAYLLO,150106.0,NaN
...,...,...,...,...,...,...,...,...,...,...
220913,2024-03-17,2020-08-09,77,MASCULINO,Criterio radiológico,AREQUIPA,AREQUIPA,ALTO SELVA ALEGRE,40102.0,NaN
220914,2024-03-17,2020-10-28,81,MASCULINO,Criterio clínico,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101.0,NaN
220915,2024-03-17,2020-05-15,68,FEMENINO,Criterio SINADEF,LIMA,LIMA,LA MOLINA,150114.0,NaN
220916,2024-03-17,2021-04-14,66,FEMENINO,Criterio clínico,LIMA,LIMA,LOS OLIVOS,150117.0,NaN


Ahora que ya tenemos los datos filtrados usando el isnull en la columna UUID.  

Podemos realizar la validación con el Departamento de  AREQUIPA:

Obtendremos:  
1. total de registros sin valores nulos 
2. total de registros con valores nulos

In [16]:
print("Total UUID sin valores nulos: ", mi_data_covid.loc[(mi_data_covid["DEPARTAMENTO"] =="AREQUIPA") & (mi_data_covid["UUID"].notnull())]["DEPARTAMENTO"].count())
print("Total UUID con valores nulos: ",mi_data_covid.loc[(mi_data_covid["DEPARTAMENTO"] =="AREQUIPA") & (mi_data_covid["UUID"].isnull())]["DEPARTAMENTO"].count())

Total UUID sin valores nulos:  10636
Total UUID con valores nulos:  59
